# MLB Stats Report: Mixture of Agents with Phidata and Groq

In this notebook, we will showcase the concept of [Mixture of Agents (MoA)](https://arxiv.org/pdf/2406.04692) using [Phidata Assistants](https://github.com/phidatahq/phidata) and [Groq API](https://console.groq.com/playground). 

The Mixture of Agents approach involves leveraging multiple AI agents, each equipped with different language models, to collaboratively complete a task. By combining the strengths and perspectives of various models, we can achieve a more robust and nuanced result. 

In our project, multiple MLB Writer agents, each utilizing a different language model (`llama3-8b-8192`, `gemma2-9b-it`, and `mixtral-8x7b-32768`), will independently generate game recap articles based on game data collected from other Phidata Assistants. These diverse outputs will then be aggregated by an MLB Editor agent, which will synthesize the best elements from each article to create a final, polished game recap. This process not only demonstrates the power of collaborative AI but also highlights the effectiveness of integrating multiple models to enhance the quality of the generated content.

### Setup

In [15]:
# Import packages
import os
import json
import statsapi
from datetime import timedelta, datetime
import pandas as pd
from phi.assistant import Assistant
from phi.llm.groq import Groq

In [16]:
api_key = os.getenv("GROQ_API_KEY")

We will configure multiple LLMs using [Phidata Assistants](https://github.com/phidatahq/phidata), each requiring a Groq API Key for access which you can create [here](https://console.groq.com/keys). These models include different versions of Meta's LLaMA 3 and other specialized models like Google's Gemma 2 and Mixtral. Each model will be used by different agents to generate diverse outputs for the MLB game recap.

In [17]:
llm_llama70b = Groq(model="llama3-70b-8192", api_key=api_key)
llm_llama8b = Groq(model="llama3-groq-8b-8192-tool-use-preview", api_key=api_key)
llm_gemma2 = Groq(model="gemma2-9b-it", api_key=api_key)
llm_mixtral = Groq(model="mixtral-8x7b-32768", api_key=api_key)

### Define Tools

First, we will define specialized tools to equip some of the agents with to assist in gathering and processing MLB game data. These tools are designed to fetch game information and player boxscores via the [MLB-Stats API](https://github.com/toddrob99/MLB-StatsAPI). By providing these tools to our agents, they can call them with relevant information provided by the user prompt and infuse our MLB game recaps with accurate, up-to-date external information.

- **get_game_info**: Fetches high-level information about an MLB game, including teams, scores, and key players.
- **get_batting_stats**: Retrieves detailed player batting statistics for a specified MLB game.
- **get_pitching_stats**: Retrieves detailed player pitching statistics for a specified MLB game.

For more information on tool use/function calling with Phidata Mixture of Agents, check out [Phidata Documentation](https://docs.phidata.com/introduction).

In [18]:
from pydantic import BaseModel, Field


class GameInfo(BaseModel):
    game_id: str = Field(description="The 6-digit ID of the game")
    home_team: str = Field(description="The name of the home team")
    home_score: str = Field(description="The score of the home team")
    away_team: str = Field(description="The name of the away team")
    away_score: str = Field(description="The score of the away team")
    winning_team: str = Field(description="The name of the winning team")
    series_status: str = Field(description="The status of the series")


def get_game_info(game_date: str, team_name: str) -> str:
    """Gets high-level information on an MLB game.

    Params:
    game_date: The date of the game of interest, in the form "yyyy-mm-dd".
    team_name: MLB team name. Both full name (e.g. "New York Yankees") or nickname ("Yankees") are valid. If multiple teams are mentioned, use the first one
    """
    sched = statsapi.schedule(start_date=game_date, end_date=game_date)
    sched_df = pd.DataFrame(sched)
    game_info_df = sched_df[sched_df["summary"].str.contains(team_name, case=False, na=False)]

    game_info = {
        "game_id": str(game_info_df.game_id.tolist()[0]),
        "home_team": game_info_df.home_name.tolist()[0],
        "home_score": game_info_df.home_score.tolist()[0],
        "away_team": game_info_df.away_name.tolist()[0],
        "away_score": game_info_df.away_score.tolist()[0],
        "winning_team": game_info_df.winning_team.tolist()[0],
        "series_status": game_info_df.series_status.tolist()[0],
    }

    return json.dumps(game_info)


def get_batting_stats(game_id: str) -> str:
    """Gets player boxscore batting stats for a particular MLB game

    Params:
    game_id: The 6-digit ID of the game
    """
    boxscores = statsapi.boxscore_data(game_id)
    player_info_df = pd.DataFrame(boxscores["playerInfo"]).T.reset_index()

    away_batters_box = pd.DataFrame(boxscores["awayBatters"]).iloc[1:]
    away_batters_box["team_name"] = boxscores["teamInfo"]["away"]["teamName"]

    home_batters_box = pd.DataFrame(boxscores["homeBatters"]).iloc[1:]
    home_batters_box["team_name"] = boxscores["teamInfo"]["home"]["teamName"]

    batters_box_df = pd.concat([away_batters_box, home_batters_box]).merge(
        player_info_df, left_on="name", right_on="boxscoreName"
    )
    batting_stats = batters_box_df[
        ["team_name", "fullName", "position", "ab", "r", "h", "hr", "rbi", "bb", "sb"]
    ].to_dict(orient="records")

    return json.dumps(batting_stats)


def get_pitching_stats(game_id: str) -> str:
    """Gets player boxscore pitching stats for a particular MLB game

    Params:
    game_id: The 6-digit ID of the game
    """
    boxscores = statsapi.boxscore_data(game_id)
    player_info_df = pd.DataFrame(boxscores["playerInfo"]).T.reset_index()

    away_pitchers_box = pd.DataFrame(boxscores["awayPitchers"]).iloc[1:]
    away_pitchers_box["team_name"] = boxscores["teamInfo"]["away"]["teamName"]

    home_pitchers_box = pd.DataFrame(boxscores["homePitchers"]).iloc[1:]
    home_pitchers_box["team_name"] = boxscores["teamInfo"]["home"]["teamName"]

    pitchers_box_df = pd.concat([away_pitchers_box, home_pitchers_box]).merge(
        player_info_df, left_on="name", right_on="boxscoreName"
    )
    pitching_stats = pitchers_box_df[["team_name", "fullName", "ip", "h", "r", "er", "bb", "k", "note"]].to_dict(
        orient="records"
    )

    return json.dumps(pitching_stats)

### Define Agents

In Phidata, Assistants are autonomous entities designed to execute a task using their Knowledge, Memory, and Tools. 

- **MLB Researcher**: Uses the `get_game_info` tool to gather high-level game information.
- **MLB Batting and Pitching Statistician**: Retrieves player batting and pitching boxscore stats using the `get_batting_stats` and `get_pitching_stats` tools.
- **MLB Writers**: Three agents, each using different LLMs (LLaMA-8b, Gemma-9b, Mixtral-8x7b), to write game recap articles.
- **MLB Editor**: Edits the articles from the writers to create the final game recap.

#### Mixture of Agents

In this demo, although the MLB Researcher and MLB Statistician agents use tool calling to gather data for the output, our Mixture of Agents framework consists of the three MLB Writer agents and the MLB Editor. This makes our MoA architecture a simple 2 layer design, but more complex architectures are possible to improve the output even more:

![Alt text](mixture_of_agents_diagram.png)

In [19]:
default_date = datetime.now().date() - timedelta(1)  # Set default date to yesterday in case no date is specified

mlb_researcher = Assistant(
    llm=llm_mixtral,
    description="An detailed accurate MLB researcher extracts game information from the user question",
    instructions=[
        "Parse the Team and date from the user question.",
        "Pass the necessary team(s) and dates to get_game_info tool",
        f"Unless a specific date is provided in the user prompt, use {default_date} as the game date",
        """
        Please include the following in your response:
            game_id: game_id
            home_team: home_team
            home_score: home_score
            away_team: away_team
            away_score: away_score
            winning_team: winning_team
            series_status: series_status
        """,
    ],
    tools=[get_game_info],
)

mlb_batting_statistician = Assistant(
    llm=llm_mixtral,
    description="An industrious MLB Statistician analyzing player boxscore stats for the relevant game",
    instructions=[
        "Given information about a MLB game, retrieve ONLY boxscore player batting stats for the game identified by the MLB Researcher",
        "Your analysis should be atleast 1000 words long, and include inning-by-inning statistical summaries",
    ],
    tools=[get_batting_stats],
)

mlb_pitching_statistician = Assistant(
    llm=llm_mixtral,
    description="An industrious MLB Statistician analyzing player boxscore stats for the relevant game",
    instructions=[
        "Given information about a MLB game, retrieve ONLY boxscore player pitching stats for a specific game",
        "Your analysis should be atleast 1000 words long, and include inning-by-inning statistical summaries",
    ],
    tools=[get_pitching_stats],
)

mlb_writer_llama = Assistant(
    llm=llm_llama70b,
    description="An experienced, honest, and industrious writer who does not make things up",
    instructions=[
        """
            Write a game recap article using the provided game information and stats.
            Key instructions:
            - Include things like final score, top performers and winning/losing pitcher.
            - Use ONLY the provided data and DO NOT make up any information, such as specific innings when events occurred, that isn't explicitly from the provided input.
            - Do not print the box score
        """,
        "Your recap from the stats should be at least 1000 words. Impress your readers!!!",
    ],
)

mlb_writer_gemma = Assistant(
    llm=llm_gemma2,
    description="An experienced and honest writer who does not make things up",
    instructions=["Write a detailed game recap article using the provided game information and stats"],
)

mlb_writer_mixtral = Assistant(
    llm=llm_mixtral,
    description="An experienced and honest writer who does not make things up",
    instructions=["Write a detailed game recap article using the provided game information and stats"],
)

mlb_editor = Assistant(
    llm=llm_llama70b,
    description="An experienced editor that excels at taking the best parts of multiple texts to create the best final product",
    instructions=["Edit recap articles to create the best final product."],
)

In [20]:
user_prompt = "write a recap of the Yankees game on July 14, 2024"

In [21]:
game_information = mlb_researcher.run(user_prompt, stream=False)

# TODO run batting and pitching stats async
batting_stats = mlb_batting_statistician.run(game_information, stream=False)
pitching_state = mlb_pitching_statistician.run(game_information, stream=False)

# TODO run mulitple writers async
stats = f"Statistical summaries for the game:\n\nBatting stats:\n{batting_stats}\n\nPitching stats:\n{pitching_state}"
llama_writer = mlb_writer_llama.run(stats, stream=False)
gemma_writer = mlb_writer_gemma.run(stats, stream=False)
mixtral_writer = mlb_writer_mixtral.run(stats, stream=False)


# Edit final outputs
editor_inputs = [llama_writer, gemma_writer, mixtral_writer]
editor = mlb_editor.run("\n".join(editor_inputs), stream=False)

print(editor)

DEBUG    *********** Assistant Run Start: b5bfcbb9-0511-4a49-9efc-ac6a50ba6e00 ***********         ]8;id=80738;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py\assistant.py]8;;\:]8;id=948958;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py#818\818]8;;\

DEBUG    Loaded memory                                                                             ]8;id=92541;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py\assistant.py]8;;\:]8;id=955453;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py#335\335]8;;\

DEBUG    Function get_game_info added to LLM.                                                           ]8;id=287289;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\base.py\base.py]8;;\:]8;id=86324;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\base.py#145\145]8;;\

DEBUG    ---------- Groq Response Start ----------                                                      ]8;id=608359;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\groq\groq.py\groq.py]8;;\:]8;id=717018;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\groq\groq.py#165\165]8;;\

DEBUG    ============== system ==============                                                         ]8;id=287441;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\message.py\message.py]8;;\:]8;id=232621;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\message.py#73\73]8;;\

...

DEBUG    Here is the edited recap article:                                                            ]8;id=699443;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\message.py\message.py]8;;\:]8;id=743197;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\message.py#79\79]8;;\
                                                                                                                   
         **Yankees Fall to Orioles 9-7 in High-Scoring Affair**                                                    
                                                                                                                   
         The New York Yankees faced off against the Baltimore Orioles on July 14, 2024, in a game                  
         marked by explosive offense and a disappointing performance from the bullpen. Despite a                   
         strong start from Carlos Rodón, the Yankees ultimately fell 9-7 to their American League                  
         East rivals.                                                                                              
                                                                                                                   
         Ben Rice and Trent Grisham were the stars of the show for the Yankees, with Rice going                    
         2-for-3 with a home run, three RBIs, and two runs scored. Grisham had an incredible day at                
         the plate, finishing 5-for-5 with two runs scored, an RBI, and two walks. Gunnar Henderson                
         also made a significant impact, going 1-for-5 with a home run, two RBIs, and a run scored.                
                                                                                                                   
         On the mound, Rodón had a solid outing, pitching four innings and allowing two earned runs                
         on three hits and three walks. He struck out five batters and kept the Orioles at bay for                 
         most of his time on the hill. However, the bullpen struggled to contain the Orioles'                      
         offense, ultimately giving up five runs in 2.1 innings of work.                                           
                                                                                                                   
         Tommy Kahnle was the first to struggle, allowing a run on one hit without recording an out                
         in the fifth inning. Michael Tonkin provided a brief respite, striking out the side in the                
         sixth, but Luke Weaver and Jake Cousins also had difficulty containing the Orioles. Clay                  
         Holmes, who entered the game in the ninth, was charged with three runs and blew the save,                 
         ultimately taking the loss.                                                                               
                                                                                                                   
         The Yankees got off to a hot start, with Rice launching a three-run homer in the first                    
         inning to give his team an early 3-0 lead. The Orioles responded with an unearned run in the              
         third, but the Yankees added to their lead with runs in the fourth and sixth innings.                     
         However, the Orioles responded with three runs in the seventh and two in the eighth to take               
         the lead, and the Yankees were unable to recover.                                                         
                                                                                                                   
         Despite the loss, there were some bright spots for the Yankees. In addition to the strong                 
         perf

DEBUG    ---------- Groq Response End ----------                                                        ]8;id=309566;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\groq\groq.py\groq.py]8;;\:]8;id=362016;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\llm\groq\groq.py#235\235]8;;\

DEBUG    --o-o-- Creating Assistant Event                                                           ]8;id=457313;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\api\assistant.py\assistant.py]8;;\:]8;id=446295;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\api\assistant.py#53\53]8;;\

DEBUG    Could not create assistant event: [WinError 10061] No connection could be made because the ]8;id=223427;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\api\assistant.py\assistant.py]8;;\:]8;id=178895;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\api\assistant.py#77\77]8;;\
         target machine actively refused it                                                                        

DEBUG    *********** Assistant Run End: 0612959d-c446-4207-98f7-d92d9efe1155 ***********           ]8;id=495590;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py\assistant.py]8;;\:]8;id=674075;file://c:\Users\jawei\lab\groq-api-cookbook\phidata-mixture-of-agents\phienv\Lib\site-packages\phi\assistant\assistant.py#962\962]8;;\